In [36]:
#Libraries
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
import plotly.express as px

In [97]:
#Dataset
vf = pd.read_csv('BulkenDognVerdier.csv', delimiter=';', on_bad_lines='skip', skiprows=[0])

vf = pl.from_pandas(vf) \
    .with_columns(
        pl.col("Tidspunkt").str.strptime(pl.Datetime,"%Y-%m-%d %H:%M:%SZ").dt.date(),
        pl.col("Vannføring").str.replace(",", ".").cast(pl.Float64)
    ) \
    .with_columns(
        iso_year = pl.col("Tidspunkt").dt.year(),
        date = pl.col("Tidspunkt").dt.strftime("%m-%d"),
    ) \
    .filter(
        pl.col("iso_year") >= 1901
    ) \
    .with_columns(
        Period = ((pl.col("iso_year")-1901)/30).cast(pl.Float64).floor(),
        Angle = (pl.col("Tidspunkt").dt.ordinal_day() / 366) * 360
    )

month_angles = [(m-1) * 30 for m in range(1, 13)]
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

vf

Tidspunkt,Vannføring,Korrigert,Kontrollert,iso_year,date,Period,Angle
date,f64,i64,i64,i32,str,f64,f64
1901-01-01,31.09981,0,3,1901,"""01-01""",0.0,0.983607
1901-01-02,21.64908,0,3,1901,"""01-02""",0.0,1.967213
1901-01-03,14.24655,0,3,1901,"""01-03""",0.0,2.95082
1901-01-04,8.681128,0,3,1901,"""01-04""",0.0,3.934426
1901-01-05,4.893484,0,3,1901,"""01-05""",0.0,4.918033
…,…,…,…,…,…,…,…
2020-12-27,26.8915,0,3,2020,"""12-27""",3.0,356.065574
2020-12-28,25.29031,0,3,2020,"""12-28""",3.0,357.04918
2020-12-29,22.84277,0,3,2020,"""12-29""",3.0,358.032787


In [104]:
import plotly.graph_objects as go

# Example data
df = vf.to_pandas()
df["Tidspunkt"] = pd.to_datetime(df["Tidspunkt"])

# Convert to angular coordinates (0–360°)
df["Angle"] = (df["Tidspunkt"].dt.dayofyear / 365) * 360

# Month tick positions
month_angles = [(m - 1) * 30 for m in range(1, 13)]
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Create figure
fig = go.Figure()

# One line per Period, with hover text
for period, group in df.groupby("Period"):
    fig.add_trace(go.Scatterpolar(
        r=group["Vannføring"],
        theta=group["Angle"],
        mode="lines+markers",  # add markers for better hover detection
        name=f"Period {period}",
        line=dict(width=2),
        opacity=0.5,
        # Custom hover label content
        text=group["Tidspunkt"].dt.strftime("%Y-%m-%d"),
        hovertemplate=(
            "<b>Date:</b> %{text}<br>" +
            "<b>Vannføring:</b> %{r:.2f}<br>" +
            "<b>Angle:</b> %{theta:.1f}°<br>" +
            "<extra></extra>"
        )
    ))

# Layout
fig.update_layout(
    title="Flood Rose (Vannføring over Tidspunkt)",
    width=900,     # ⬅️ wider
    height=900,    # ⬅️ taller
    polar=dict(
        radialaxis=dict(title="Vannføring", showgrid=True),
        angularaxis=dict(
            tickvals=month_angles,
            ticktext=month_labels,
            rotation=90,
            direction="counterclockwise",
            showgrid=True
        )
    ),
    showlegend=True
)

fig.show()

In [110]:
import plotly.graph_objects as go
from ipywidgets import Output
import pandas as pd

# --- Sample setup ---
df = vf.to_pandas()
df["Tidspunkt"] = pd.to_datetime(df["Tidspunkt"])
df["Angle"] = (df["Tidspunkt"].dt.dayofyear / 365) * 360

month_angles = [(m - 1) * 30 for m in range(1, 13)]
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# --- Build FigureWidget ---
fig = go.FigureWidget()

for period, group in df.groupby("Period"):
    fig.add_trace(go.Scatterpolar(
        r=group["Vannføring"],
        theta=group["Angle"],
        mode="lines+markers",
        name=f"Period {period}",
        line=dict(width=2),
        opacity=0.3,
        text=group["Tidspunkt"].dt.strftime("%Y-%m-%d"),
        hovertemplate="<b>Date:</b> %{text}<br>Vannføring: %{r:.2f}<extra></extra>"
    ))

fig.update_layout(
    title="Flood Rose (Hover Highlight Demo)",
    width=900, height=900,
    polar=dict(
        radialaxis=dict(title="Vannføring", showgrid=True),
        angularaxis=dict(
            tickvals=month_angles,
            ticktext=month_labels,
            rotation=90,
            direction="counterclockwise"
        )
    ),
    showlegend=True
)

# --- Hover interaction ---
def handle_hover(trace, points, state):
    # Dim all traces
    for t in fig.data:
        t.opacity = 0.1
    # Highlight hovered trace
    trace.opacity = 1.0

def handle_unhover(trace, points, state):
    # Restore all traces
    for t in fig.data:
        t.opacity = 0.3

# Attach hover handlers to each trace
for tr in fig.data:
    tr.on_hover(handle_hover)
    tr.on_unhover(handle_unhover)

fig


FigureWidget({
    'data': [{'hovertemplate': '<b>Date:</b> %{text}<br>Vannføring: %{r:.2f}<extra></extra>',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'Period 0.0',
              'opacity': 0.3,
              'r': {'bdata': ('U9DtJY0ZP0DZfFwbKqY1QFioNc07fi' ... 'XNO34sQItPATCe+TBA3rBtUWY7NUA='),
                    'dtype': 'f8'},
              'text': array(['1901-01-01', '1901-01-02', '1901-01-03', ..., '1930-12-29',
                             '1930-12-30', '1930-12-31'], shape=(10957,), dtype=object),
              'theta': {'bdata': ('/fjx48eP7z/9+PHjx4//P7169erVqw' ... 'w4cGB2QAQHDhw4cHZAAAAAAACAdkA='),
                        'dtype': 'f8'},
              'type': 'scatterpolar',
              'uid': 'acb4300b-89a0-4b15-9297-95bf8088e428'},
             {'hovertemplate': '<b>Date:</b> %{text}<br>Vannføring: %{r:.2f}<extra></extra>',
              'line': {'width': 2},
              'mode': 'lines+markers',
              'nam